In [2]:
! pip install cleantext

DEPRECATION: Loading egg at /home/siddhantp/anaconda3/lib/python3.11/site-packages/ianvs-0.1.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [3]:
import requests
import pandas as pd
import json
import datetime
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from cleantext import clean

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/siddhantp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
mental_health_keywords = [
    "depression", "depressed", "anxiety", "suicidal", 
    "suicide", "addiction", "substance abuse", "overwhelmed", 
    "hopeless", "self harm", "bipolar", "mental health", 
    "therapy", "crisis", "panic attack"]

In [6]:
# Using the Reddit API credentials
CLIENT_ID = 'xFa6r8x1ww7TGMAz2mWxqw'
CLIENT_SECRET = 'tTrSXTuXXsMFEf8h6BEW9ItQCaIcpg'
USER_AGENT = 'mental_health_analysis_script/1.0'

# Authenticating with Reddit API
auth = requests.auth.HTTPBasicAuth(CLIENT_ID, CLIENT_SECRET)
data = {
    'grant_type': 'password',
    'username': 'infinitely_Sid',
    'password': '********'
}
headers = {'User-Agent': USER_AGENT}
res = requests.post('https://www.reddit.com/api/v1/access_token', auth=auth, data=data, headers=headers)
TOKEN = res.json()['access_token']
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

In [7]:

# Function to extract posts by keyword
def get_reddit_posts(keyword, limit=100):
    posts = []
    url = f"https://oauth.reddit.com/search?q={keyword}&sort=new&limit={limit}"
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        for post in data['data']['children']:
            post_data = post['data']
            if any(kw.lower() in post_data['title'].lower() or 
                   (post_data['selftext'] and kw.lower() in post_data['selftext'].lower()) 
                   for kw in mental_health_keywords):
                posts.append({
                    'post_id': post_data['id'],
                    'timestamp': datetime.datetime.fromtimestamp(post_data['created_utc']).strftime('%Y-%m-%d %H:%M:%S'),
                    'title': post_data['title'],
                    'content': post_data['selftext'],
                    'subreddit': post_data['subreddit'],
                    'upvotes': post_data['ups'],
                    'comments': post_data['num_comments'],
                    'url': f"https://www.reddit.com{post_data['permalink']}"
                })
    return posts


In [11]:
# Collecting posts for each keyword
all_posts = []
for keyword in mental_health_keywords:
    print(f"Collecting posts for keyword: {keyword}")
    keyword_posts = get_reddit_posts(keyword)
    all_posts.extend(keyword_posts)

# Removing duplicates based on post_id
unique_posts = {post['post_id']: post for post in all_posts}.values()
posts_df = pd.DataFrame(list(unique_posts))

# Preprocessing text data
def preprocess_text(text):
    if not isinstance(text, str) or pd.isna(text) or text.strip() == "":
        return ""
    
    # comprehensive cleaning
    try:
        cleaned_text = clean(text,
                         extra_spaces=True,
                         lowercase=True,
                         numbers=True,
                         punct=True,
                         stopwords=True,
                         stp_lang='english')
        return cleaned_text
    except Exception as e:
        # Fallback cleaning if cleantext fails
        print(f"Warning: Cleaning failed for text: {text[:30]}... Error: {str(e)}")
        return "" if pd.isna(text) else text.lower().strip()
    

In [12]:
# preprocessing to title and content
posts_df['cleaned_title'] = posts_df['title'].apply(preprocess_text)
posts_df['cleaned_content'] = posts_df['content'].apply(preprocess_text)

# Saving to CSV and JSON
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
csv_filename = f"mental_health_posts_{timestamp}.csv"
json_filename = f"mental_health_posts_{timestamp}.json"

posts_df.to_csv(csv_filename, index=False)
posts_df.to_json(json_filename, orient='records')

print(f"Collected {len(posts_df)} unique posts related to mental health")
print(f"Data saved to {csv_filename} and {json_filename}")

Collected 922 unique posts related to mental health
Data saved to mental_health_posts_20250323_121040.csv and mental_health_posts_20250323_121040.json
